In [ ]:
import os
from langchain.chat_models import init_chat_model
import re
import numpy as np
import openai
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import Annotated
from typing import Annotated
from typing_extensions import TypedDict
from datetime import datetime
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.checkpoint.memory import InMemorySaver  # Add this import
from langgraph.prebuilt import ToolNode, tools_condition  # Add this import
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnableConfig
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


os.environ["OPENAI_API_KEY"] = ""

llm = init_chat_model("openai:gpt-4.1")

In [10]:
%pip install -qU langchain-unstructured

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Gus Agung\\Desktop\\backend-hotel-agent\\.venv\\Lib\\site-packages\\~ydantic_core\\_pydantic_core.cp312-win_amd64.pyd'
Check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Building the  ingestion and retrieval 

In [3]:
file_path = (
    "./assets/Puri Santrian Hotel – Overview & Contact Information.pdf"
)

In [ ]:


loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)



pages

[Document(metadata={'producer': 'WeasyPrint 65.1', 'creator': 'ChatGPT', 'creationdate': '', 'title': 'Puri Santrian Hotel – Overview & Contact Information', 'author': 'ChatGPT Deep Research', 'source': './assets/Puri Santrian Hotel – Overview & Contact Information.pdf', 'total_pages': 10, 'page': 0, 'page_label': '1'}, page_content='Puri Santrian Hotel – Overview & Contact\nInformation\nPuri Santrian is a 4-star Balinese beach resort located at the tranquil southern end of Sanur Beach in Bali,\nIndonesia . The resort is  family-owned and operated, designed like an extended Balinese royal\ncompound with separate wings for different room categories. Set amid lush tropical gardens, it offers\n199 guest rooms across low-rise buildings (up to 3 floors) and traditional bungalows. The atmosphere\nis upscale yet culturally rooted, aiming to preserve Balinese traditions while providing modern comforts.\nGuests can enjoy direct beachfront access with sunrise views, multiple swimming pools, and 

In [ ]:
# Create vector store from your documents
vector_store = InMemoryVectorStore.from_documents(pages, OpenAIEmbeddings())

# Create a retrieval chain
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0),  # Use OpenAI's chat model
    chain_type="stuff",  # Simple chain that stuffs all docs into prompt
    retriever=vector_store.as_retriever(search_kwargs={"k": 3}),  # Get top 3 relevant docs
    return_source_documents=True,  # Return the source docs used
)

# Ask a question
question = "do you guys offer free parking? and also what restaurant do you guys have?"
response = qa_chain.invoke({"query": question})

print(f"Question: {question}")
print(f"Answer: {response['result']}")
print("\nSources used:")
for doc in response['source_documents']:
    print(f"- Page {doc.metadata['page']}: {doc.page_content[:200]}...")

Question: do you guys offer free parking? and also what restaurant do you guys have?
Answer: Yes, the resort offers complimentary on-site parking for guests. The parking area is within the hotel grounds, and there is security staff monitoring it 24/7. The parking lot can accommodate standard cars but not large buses. Valet service is not formal, but staff will assist if needed.

Regarding restaurants, Puri Santrian has two main restaurants on-site:
1. Beach Club Restaurant: This open-air beachfront restaurant serves international and Indonesian cuisine. It is open daily from 6:00 AM to midnight and offers a buffet breakfast, lunch, and dinner with themed dinner events some nights.
2. Mezzanine Asian Kitchen & Bar: A specialty restaurant near the hotel entrance offering a refined dinner experience with a focus on Asian cuisines, especially Japanese and Chinese dishes. It is open in the evenings from 6:00 PM to 11:00 PM. Reservations are recommended for Mezzanine, especially in high seas

In [39]:
docs = vector_store.similarity_search("what is the hotel name", k=2)
for doc in docs:
    print(f"Page {doc.metadata['page']}: {doc.page_content[:300]}\n")

Page 7: Medical Assistance: A doctor is on call 24/7 for the resort. If you feel unwell or need medical
help, contact the front desk and they can arrange a doctor visit to the hotel or assist in transporting
you to an international clinic or hospital nearby. The staff is trained in basic first aid and the h

Page 0: Puri Santrian Hotel – Overview & Contact
Information
Puri Santrian is a 4-star Balinese beach resort located at the tranquil southern end of Sanur Beach in Bali,
Indonesia . The resort is  family-owned and operated, designed like an extended Balinese royal
compound with separate wings for different 



# Lets solve the multi query issues first

In [ ]:
 its already getting the right data answers most of the time but it needs to be more accurate maybe what we can do is add sql to it 
add an sql id to each one for further information in the metada

# Tools creation 

## RAG for hotels and query decomposition

In [ ]:
# need to add decompositions to the retriever agent

from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import numpy as np

class VectorStoreRetriever:
    def __init__(self, docs: list, oai_client):
        self._docs = docs
        self._client = oai_client
        # Create vector store from documents
        self._vector_store = InMemoryVectorStore.from_documents(
            docs, 
            OpenAIEmbeddings()
        )
        
        # Initialize decomposition components
        self._llm = ChatOpenAI(temperature=0)
        self._setup_decomposition()
    
    def _setup_decomposition(self):
        """Setup the decomposition chain using external prompt file"""
        # Load prompt from external file
        with open('assets/prompts/rag_decomposition.txt', 'r') as f:
            template = f.read()
        
        # Create decomposition prompt template
        prompt_decomposition = ChatPromptTemplate.from_template(template)
        
        # Create decomposition chain
        self._decomposition_chain = (
            prompt_decomposition | 
            self._llm | 
            StrOutputParser() | 
            (lambda x: [q.strip() for q in x.split("\n") if q.strip()])
        )
    
    @classmethod
    def from_documents(cls, docs: list, oai_client):
        """Alternative constructor to create instance from documents"""
        return cls(docs, oai_client)
    
    def decompose_question(self, question: str):
        """Decompose a complex question into sub-questions"""
        try:
            sub_questions = self._decomposition_chain.invoke({"question": question})
            return sub_questions
        except Exception as e:
            print(f"Error in decomposition: {e}")
            return [question]  # Fallback to original question
    
    def retrieve_with_decomposition(self, question: str, k: int = 3):
        """Retrieve documents using question decomposition for better coverage"""
        # Decompose the question
        sub_questions = self.decompose_question(question)
        print(f"Original question: {question}")
        print(f"Decomposed into {len(sub_questions)} sub-questions:")
        for i, sq in enumerate(sub_questions, 1):
            print(f"  {i}. {sq}")
        print()
        
        # Retrieve documents for each sub-question
        all_docs = []
        for sub_q in sub_questions:
            docs = self.similarity_search(sub_q, k=k)
            all_docs.extend(docs)
        
        # Remove duplicates based on page content
        unique_docs = []
        seen_content = set()
        for doc in all_docs:
            content_hash = hash(doc.page_content[:100])  # Hash first 100 chars
            if content_hash not in seen_content:
                seen_content.add(content_hash)
                unique_docs.append(doc)
        
        return unique_docs
    
    def similarity_search(self, query: str, k: int = 3):
        """Direct access to vector store similarity search"""
        return self._vector_store.similarity_search(query, k=k)


In [89]:
@tool
def lookup_hotel_info(query: str) -> tuple[str, list[str]]:
    """Consult the hotel information to answer questions about amenities, services, policies, facilities, dining options, accommodations, and guest experiences.
    Use this to provide accurate information about the hotel based on available documentation.
    
    Returns:
        tuple: (formatted_response, list_of_document_strings)
    """
    
    retriever = VectorStoreRetriever.from_documents(pages, llm)
    docs = retriever.retrieve_with_decomposition(query, k=3)
    
    if not docs:
        return "No relevant information found for your query.", []
    
    # Create a list of document content strings
    doc_strings = [doc.page_content for doc in docs]
    
    # Format response with clear sections
    response = f"Based on the hotel documentation, here's what I found for: '{query}'\n\n"
    
    for i, doc in enumerate(docs, 1):
        response += f"Source {i} (Page {doc.metadata['page']}):\n{doc.page_content}\n\n"
    
    return response.strip(), doc_strings

rag_hotel_info = lookup_hotel_info

In [71]:
tools = [rag_hotel_info]
llm.bind_tools(tools)

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000275E3D7BEC0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000275E542CFB0>, root_client=<openai.OpenAI object at 0x00000275DF0E6E40>, root_async_client=<openai.AsyncOpenAI object at 0x00000275E541A330>, model_name='gpt-4.1', model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'lookup_hotel_info', 'description': 'Consult the hotel information to answer questions about amenities, services, policies, facilities, dining options, accommodations, and guest experiences.\n    Use this to provide accurate information about the hotel based on available documentation.', 'parameters': {'properties': {'query': {'type': 'string'}}, 'required': ['query'], 'type': 'object'}}}]}, config={}, config_factories=[])

# Langgraph Agent

In [101]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


In [106]:
# ----- graph state -----
class GraphState(TypedDict):
    question: str
    docs: List[str]         # raw snippets from your tool
    notes: str              # the formatted string your tool returns
    answer: str             # final answer from llm (or fallback)

# ----- nodes -----
def retrieve_node(state: GraphState) -> GraphState:
    q = state["question"]
    notes, docs = rag_hotel_info(q)  # your tool
    return {"question": q, "docs": docs or [], "notes": notes, "answer": state.get("answer","")}

# Load prompt from AI_concierge.txt
def load_prompt_template(file_path: str) -> str:
    with open(file_path, 'r') as f:
        return f.read()

# Load the hotel concierge prompt from file
generate_prompt_template = load_prompt_template('assets/prompts/AI_concierge.txt')

# Create the prompt template - make sure it uses {question} and {context} placeholders
generate_prompt = PromptTemplate.from_template(generate_prompt_template)
generate_chain = generate_prompt | llm | StrOutputParser()

def generate_node(state: GraphState) -> GraphState:
    q = state["question"]
    docs = state["docs"]
    if not docs:
        return {**state, "answer": "I don't have that in the hotel docs yet."}
    
    # Use all documents without capping
    context = "\n---\n".join(docs)
    
    out = generate_chain.invoke({"question": q, "context": context})
    return {**state, "answer": out}

# ----- wire graph -----
workflow = StateGraph(GraphState)
workflow.add_node("retrieve", retrieve_node)
workflow.add_node("generate", generate_node)
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate")
app = workflow.compile()

In [116]:
inputs = {"question": "do you have any recommendations for an asian restaurant in the area?"}
for step in app.stream(inputs):
    for k in step:
        print("finished:", k)
    # final answer
print("\nANSWER:", step["generate"]["answer"])

Original question: do you have any recommendations for an asian restaurant in the area?
Decomposed into 3 sub-questions:
  1. 1. What are the top-rated Asian restaurants near the hotel?
  2. 2. Are there any specific Asian cuisines that you prefer or are looking for in the area?
  3. 3. Do you have any dietary restrictions or preferences that we should consider when recommending an Asian restaurant?

finished: retrieve
finished: generate

ANSWER: Certainly! At Hotel Puri Santrian, we have the Mezzanine Asian Kitchen & Bar, which offers a refined dining experience with a focus on Asian cuisines, especially Japanese and Chinese dishes. The restaurant features a two-level semi-fine dining space and a menu that includes sushi, teppanyaki, Chinese delicacies, and other Pan-Asian creations prepared with market-fresh ingredients. Reservations are recommended, especially during high season. The restaurant also has an elegant bar serving Asian-inspired cocktails, sake, and a curated wine list t